Author-VIRAJ RANADE
CV-24678 Project
This Notebook is made as a toolbox to try and run the results of our Analysis. These networks will be used on the pre-processed Images to obtain a result.

In [1]:
import numpy as np
import torch
import cv2
import PIL
import glob
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

In [2]:
normal='/content/drive/MyDrive/CV_project/normal/*.*'
osteoporosis='/content/drive/MyDrive/CV_project/osteoporosis/*.*'

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [4]:
def create_dataset(path1,path2):
  set_1=glob.glob(path1)
  set_2=glob.glob(path2)

  s1=[]
  s2=[]
  lab1=np.zeros((186))
  lab2=np.ones((186))

  for i in set_1:
    temp1=cv2.imread(i)
    gray1=cv2.cvtColor(temp1,cv2.COLOR_BGR2GRAY)
    gray_scaled1=cv2.resize(gray1,(156,235)).flatten()
    s1.append(gray_scaled1)
    #lab.append(0)

  for i in set_2:
    temp2=cv2.imread(i)
    gray2=cv2.cvtColor(temp2,cv2.COLOR_BGR2GRAY)
    gray_scaled2=cv2.resize(gray2,(156,235)).flatten()
    s2.append(gray_scaled2)
    #lab.append(1)

  s1=np.asarray(s1)
  s2=np.asarray(s2)
  data_2=np.concatenate((s1,s2),axis=0)
  lab=np.concatenate((lab1,lab2),axis=0)
  print(type(data_2))
  return data_2,lab


In [ ]:
data_20,labels_20=create_dataset(normal,osteoporosis)
print(data_20.shape)
print(labels_20.shape)

In [ ]:
import sklearn
import sklearn.decomposition as dec
from sklearn import model_selection
train_2,test_2,train_labels_2,test_labels_2=sklearn.model_selection.train_test_split(data_20,labels_20,test_size=0.15)

In [ ]:
class MyDataSet(Dataset):
  def __init__(self,x,y):
    self.x=x
    self.y=y
    #self.length=np.shape(self.data)[0]
    self.length=len(self.x)
    print(self.length)

  def __len__(self):
    return self.length

  def __getitem__(self,index):
    xx=torch.FloatTensor(self.x[index])
    yy=torch.FloatTensor(self.y[index])
    return xx,yy

In [ ]:
train_set=MyDataSet(train_2,train_labels_2.reshape(316,1))
train_args=dict(shuffle=True,batch_size=32,num_workers=1,collate_fn=None)
train_loader=DataLoader(train_set,**train_args)

val_set=MyDataSet(test_2,test_labels_2.reshape(56,1))
val_args=dict(shuffle=False,batch_size=32,num_workers=1,collate_fn=None)
val_loader=DataLoader(val_set,**val_args)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [ ]:
class MLP(nn.Module):
  def __init__(self,input_size,hiddens,output_size):
    super(MLP,self).__init__()
    Linear_layers=[nn.Linear(input_size,hiddens[0],bias=True),nn.BatchNorm1d(hiddens[0]),nn.ReLU(inplace=True),nn.Dropout(0.5)]
    for i in range(len(hiddens)-1):
      Linear_layers.append(nn.Linear(hiddens[i],hiddens[i+1],bias=True))
      Linear_layers.append(nn.BatchNorm1d(hiddens[i+1]))
      Linear_layers.append(nn.ReLU(inplace=True))
      Linear_layers.append(nn.Dropout(0.5))
    Linear_layers.append(nn.Linear(hiddens[i+1],output_size))

    self.layers=nn.Sequential(*Linear_layers)

  def forward(self,x):
    return self.layers(x)

In [ ]:
input=47
hiddens=[512,1024,512,256,128]
output=2
steps=[x for x in np.arange(10)*5]
print(steps)

In [ ]:
model=MLP(input,hiddens,output)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=1e-3,weight_decay=1e-4)
scheduler=optim.lr_scheduler.MultiStepLR(optimizer,steps)

device=torch.device('cuda' if torch.cuda else 'cpu')
model=model.to(device)
print(model)

In [ ]:
class Block(nn.Module):
  def __init__(self,in_channels,hidden_channels,out_channels,kernel_size=3,stride=1):
    self.stride=stride
    super().__init__()

    self.in_channel=in_channels
    self.hidden_channels=hidden_channels
    self.out_channels=out_channels

    self.b=nn.Sequential(nn.Conv2d(self.in_channel,self.hidden_channels),nn.BatchNorm2d(self.hidden_channels),nn.ReLU,
                         nn.Conv2d(self.hidden_channels,self.hidden_channels),nn.BatchNorm2d(self.hidden_channels),nn.ReLU,
                         nn.Conv2d(self.hidden_channels,self.out_channels),nn.BatchNorm2d(self.out_channels))
    
    if self.stride==1:
      self.s=nn.Identity()
    else:
      self.s=nn.Sequential(nn.Conv2d(self.in_channel,self.out_channels,kernel_size=3,stride=self.stride,padding=1),nn.BatchNorm2d(self.out_channels))
    
  def forward(self,x,flag):

    embedding=self.b(x)
    shortcut=self.s(x)
    if flag:
      self.out=nn.ELU(embedding+shortcut)
    else:
      self.out=nn.ReLU(embedding+shortcut)
    
    return self.out

In [ ]:
class CNN(nn.Module):
  def __init__(self,in_features,num_classes,feat_dim=1):
    super().__init__()
    self.convolutional_layers=nn.Sequential(
        nn.Conv2d(in_features,64,kernel_size=3,stride=2,bias=False),
        nn.BatchNorm2d(64),
        nn.ReLU(),

        Block(64,128,64),
        Block(64,128,64),
        Block(64,128,128,3,2),
        Block(128,256,128),
        Block(128,256,128),
        Block(128,256,256,3,2),
        Block(256,512,512),
        Block(512,1024,1024,3,2),

        nn.AdaptiveAvgPool2d((1,1)),
        nn.Flatten()
    )

    self.linear_layers=nn.Sequential(nn.Linear(1024,512),nn.BatchNorm1d(512),nn.ReLU(inplace=True),nn.Dropout(0.5),
                nn.Linear(512,256),nn.BatchNorm1d(256),nn.ReLU(inplace=True),nn.Dropout(0.5),
                nn.Linear(256,128),nn.BatchNorm1d(128),nn.ReLU(inplace=True),nn.Dropout(0.5),
                nn.Linear(128,32),nn.BatchNorm1d(32),nn.ReLU(inplace=True),nn.Dropout(0.5),
                nn.Linear(32,2)
    )

  def forward(self,x,return_embedding=False):
    embedding=self.convolutional_layers(x)
    output=self.linear_layers(embeddings)
    if return_embedding:
      return embedding,output
    else:
      return output

In [ ]:
network=CNN(47,2,1)
network.to(device)

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=1e-3,weight_decay=1e-4)
scheduler=optim.lr_scheduler.MultiStepLR(optimizer,steps)

print(network)